# AI 이용 주식 종목 추천

In [16]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

import yfinance as yf
import pandas as pd

In [22]:
import FinanceDataReader as fdr

# 주식 종목 리스트 (예: 삼성전자, 에코프로 등)
stocks = [('005930.KS', '삼성전자'), ('086520.KQ', '에코프로'), ('003490.KS', '대한항공'), ('373220.KS', 'LG엔솔'), 
          ('207940.KS', '삼성바이오로직스'), ('012450.KS', '한화에어로스페이스')] 

# 주식 데이터 다운로드 (yfinance를 사용)
def download_stock_data(ticker, start_date="2024-01-01"):
    stock_data = fdr.DataReader(ticker[0][:6], start_date)  #셀트리온
    return stock_data

# 주식 데이터 수집
stock_data = {}
for stock in stocks:
    stock_data[stock] = download_stock_data(stock)

stock_data

{('005930.KS',
  '삼성전자'):              Open   High    Low  Close    Volume    Change
 Date                                                      
 2024-01-02  78200  79800  78200  79600  17142847  0.014013
 2024-01-03  78500  78800  77000  77000  21753644 -0.032663
 2024-01-04  76100  77300  76100  76600  15324439 -0.005195
 2024-01-05  76700  77100  76400  76600  11304316  0.000000
 2024-01-08  77000  77500  76400  76500  11088724 -0.001305
 ...           ...    ...    ...    ...       ...       ...
 2024-11-11  56700  56800  55000  55000  29811326 -0.035088
 2024-11-12  54600  54600  53000  53000  37962881 -0.036364
 2024-11-13  52000  53000  50500  50600  52527995 -0.045283
 2024-11-14  50200  51800  49900  49900  48510715 -0.013834
 2024-11-15  50300  54200  50300  53500  43748624  0.072144
 
 [214 rows x 6 columns],
 ('086520.KQ',
  '에코프로'):               Open    High     Low   Close   Volume    Change
 Date                                                         
 2024-01-02  1295

In [23]:
# 상승 가능성 판단을 위한 간단한 분석 (예: 이동평균 분석)
def calculate_moving_averages(stock_data):
    stock_data['SMA20'] = stock_data['Close'].rolling(20).mean()
    stock_data['SMA50'] = stock_data['Close'].rolling(50).mean()
    # 현재 SMA20 > SMA50인 경우 상승 가능성 있다고 판단
    # return stock_data['SMA20'].iloc[-1] > stock_data['SMA50'].iloc[-1] 
    return True

# 상승 가능성 높은 종목 리스트
potential_rising_stocks = []
for stock, data in stock_data.items():
    if calculate_moving_averages(data):
        potential_rising_stocks.append(stock)

potential_rising_stocks

[('005930.KS', '삼성전자'),
 ('086520.KQ', '에코프로'),
 ('003490.KS', '대한항공'),
 ('373220.KS', 'LG엔솔'),
 ('207940.KS', '삼성바이오로직스'),
 ('012450.KS', '한화에어로스페이스')]

In [24]:
import yfinance as yf

# financials 딕셔너리 생성
financials = {}

# 상승 가능성이 있는 주식들의 티커 리스트(potential_rising_stocks)에서 각 주식의 재무 상태를 가져옵니다.
for ticker in potential_rising_stocks:
    # 각 티커에 해당하는 주식 데이터를 yfinance에서 불러옵니다.
    stock = yf.Ticker(ticker[0])
    
    # 해당 주식의 재무 정보를 저장할 딕셔너리 구조를 생성합니다.
    financials[ticker] = {}
    
    # 손익계산서(Income Statement) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['income statement'] = stock.financials
    
    # 대차대조표(Balance Sheet) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['balance sheet'] = stock.balance_sheet
    
    # 현금흐름표(Cash Flow Statement) 데이터를 가져와서 financials 딕셔너리에 저장합니다.
    financials[ticker]['cash flow'] = stock.cashflow

# 저장된 financials 딕셔너리의 키(티커 목록)를 출력합니다.
print(financials.keys())

# 전체 재무 데이터를 출력합니다.
financials

dict_keys([('005930.KS', '삼성전자'), ('086520.KQ', '에코프로'), ('003490.KS', '대한항공'), ('373220.KS', 'LG엔솔'), ('207940.KS', '삼성바이오로직스'), ('012450.KS', '한화에어로스페이스')])


{('005930.KS',
  '삼성전자'): {'income statement':                                                            2023-12-31  \
  Tax Effect Of Unusual Items                            -22864080000.0   
  Tax Rate For Calcs                                               0.24   
  Normalized EBITDA                                    50698344000000.0   
  Total Unusual Items                                    -95267000000.0   
  Total Unusual Items Excluding Goodwill                 -95267000000.0   
  Net Income From Continuing Operation Net Minori...   14473401000000.0   
  Reconciled Depreciation                              38666559000000.0   
  Reconciled Cost Of Revenue                          180388580000000.0   
  EBITDA                                               50603077000000.0   
  EBIT                                                 11936518000000.0   
  Net Interest Income                                   3427769000000.0   
  Interest Expense                                     

In [25]:
# OpenAI API로 상승 가능성 높은 종목에 대한 분석 요청
# 분석할 주식 리스트와 재무 상태를 기반으로 프롬프트를 작성합니다.
prompt = f"다음 주식들은 기술적 지표에 기반하여 상승 가능성이 있습니다: {', '.join(str(potential_rising_stocks))}. " + \
         "각 주식의 재무 상태는 다음과 같습니다:\n"

# 상승 가능성이 높은 종목의 재무 상태를 프롬프트에 추가합니다.
for ticker in potential_rising_stocks:
    prompt += f"\n{ticker}의 재무 상태:\n{financials[ticker]}\n"

# 프롬프트에 주식 추천 요청을 추가합니다.
prompt += "이 주식들 중에 어떤 주식이 가장 투자에 유망한지 두 종목을 추천해 주고 그 이유를 설명해 주세요. 한글로 답해주세요."

# OpenAI API로 질문을 보내고, 분석 결과를 받습니다.
response = client.chat.completions.create(
    model=Model,  # 사용할 모델을 지정합니다.
    messages=[
        {"role": "system", "content": "You are a helpful investment professional."},  # 시스템 역할 정의
        {"role": "user", "content": prompt},  # 사용자의 질문을 프롬프트로 전달
    ]
)

# OpenAI로부터 받은 응답을 출력합니다.
print(response.choices[0].message.content)

두 종목을 추천드리겠습니다: 

1. **삼성전자 (005930.KS)**
   - **이유**: 삼성전자는 안정적인 재무 상태를 보여주고 있으며, 앞으로의 시장 전망 또한 긍정적입니다. 2023년 기준으로净 수익은 15487100000000.0원에 달하며, 총 매출도 258935494000000.0원으로 안정적인 수익 구조를 갖추고 있습니다. 최근 기술적 지표 분석에서도 상승 가능성이 제기되고 있으며, 반도체부터 전자기기까지 폭넓은 제품군을 보유하고 있어 다변화된 수익원이 큰 장점입니다. 또한 글로벌 시장에서 확고한 위치를 차지하고 있는 점도 긍정적인 요소입니다.

2. **에코프로 (086520.KQ)**
   - **이유**: 에코프로는 전기차 배터리 및 기타 관련 분야의 강력한 성장 가능성을 가진 기업입니다. 2023년 기준으로净 수익은 51016615530원으로, टोटल 매출도 7260200345720.0원이 보이고 있습니다. 전 세계의 친환경 트렌드와 전기차 수요 증가에 힘입어 빠르게 성장하고 있으며, 향후 배터리 소재 및 재활용 분야에서 시장 점유율을 확대할 가능성이 높습니다. 기술적 지표에 따라서도 상승 가능성이 있는 주식입니다.

이 두 종목은 각각의 산업에서 강력한 위치를 점하고 있으며, 안정적인 재무 상태와 성장 가능성 덕분에 투자에 매력적인 선택이 될 것입니다.
